In [1]:
!pip install xgboost==1.0.2
!pip install tensorflow==1.13.1
!pip install seaborn==0.9.0
!pip install eli5==0.10.1
!pip install ann-visualizer==2.5
!pip install graphviz==0.13.2
!pip install h5py==2.9.0
!pip install imbalanced-learn==0.6.1
!pip install imblearn==0.0
!pip install Keras==2.2.4
!pip install scipy==1.4.1
!pip install scikit-learn==0.22.1
!pip install tensorflow==1.13.1
!pip install pickleshare==0.7.5
!pip install Pillow==6.2.0
!pip install statsmodels==0.10.1
!pip install Cython==0.29.13
!pip install pydotplus==2.0.2
!pip install python-dateutil==2.8.0
!pip install tqdm==4.43.0
!pip install zipp==0.6.0

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_score, recall_score ,confusion_matrix , f1_score ,roc_curve,roc_auc_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm1


In [2]:
import os

In [3]:
os.listdir()

['.ipynb_checkpoints',
 'catboost_info',
 'DataDictionaryWiDS2021.csv',
 'Datathon_file _average.ipynb',
 'Datathon_file.ipynb',
 'lightgbm_subset.ipynb',
 'Predictions_stacking_now.csv',
 'Predictions_subset.csv',
 'rekha_stacking.ipynb',
 'SampleSubmissionWiDS2021.csv',
 'SolutionTemplateWiDS2021.csv',
 'starter-s-notebook-with-lgbm-classifier.ipynb',
 'TrainingWiDS2021.csv',
 'UnlabeledWiDS2021.csv',
 'Untitled.ipynb',
 'Untitled1.ipynb',
 'widsdatathon2021.zip']

In [4]:
df= pd.read_csv('TrainingWiDS2021.csv')
test_data = pd.read_csv('UnlabeledWiDS2021.csv')

In [5]:
df.shape
test_data

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,...,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
0,1,144740,10141,72,NaN,0,Caucasian,F,152.4,Floor,...,NaN,NaN,NaN,0,0,0,0,0,0,0
1,2,141990,10141,86,NaN,0,Caucasian,F,175.3,Emergency Department,...,NaN,NaN,NaN,0,0,0,0,0,0,0
2,3,142038,10141,72,NaN,0,Caucasian,F,162.6,Floor,...,NaN,NaN,NaN,0,0,0,0,0,0,0
3,4,138628,10141,66,NaN,0,Caucasian,M,177.8,Floor,...,NaN,NaN,NaN,0,0,0,0,0,0,0
4,5,141682,10141,89,NaN,0,Caucasian,M,170.2,Direct Admit,...,NaN,NaN,NaN,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10229,10230,143750,10140,36,37.5,0,Caucasian,F,170.1,Floor,...,NaN,NaN,NaN,0,0,0,0,0,0,0
10230,10231,143813,10140,61,32.1,0,Caucasian,F,160.0,Emergency Department,...,NaN,NaN,NaN,0,0,0,0,0,0,0
10231,10232,137126,10140,74,22.7,0,Caucasian,F,165.1,Step-Down Unit (SDU),...,NaN,NaN,NaN,0,0,0,0,0,0,0
10232,10233,135652,10140,90,19.9,0,Caucasian,F,160.0,Emergency Department,...,NaN,NaN,NaN,0,0,0,0,0,0,0


In [6]:
df.columns

Index(['Unnamed: 0', 'encounter_id', 'hospital_id', 'age', 'bmi',
       'elective_surgery', 'ethnicity', 'gender', 'height',
       'hospital_admit_source',
       ...
       'h1_pao2fio2ratio_max', 'h1_pao2fio2ratio_min', 'aids', 'cirrhosis',
       'hepatic_failure', 'immunosuppression', 'leukemia', 'lymphoma',
       'solid_tumor_with_metastasis', 'diabetes_mellitus'],
      dtype='object', length=181)

In [7]:
test_data.shape

(10234, 180)

In [8]:
removal_category_col=[]

In [9]:
for col in df.columns:
    if df[col].dtype=='object':
        print(col)
        print(len(df[col]))
        print('Null records ')
        print(df[col].isnull().sum(axis=0))
        print('\n')
        print(((df[col].isnull().sum(axis=0)) / len(df[col])) * 100)
        if ((df[col].isnull().sum(axis=0)) / len(df[col])) * 100 > 50:
            removal_category_col.append(col)    
        print('Non Null records')
        print(len(df[col]) - df[col].isnull().sum(axis=0))
        print('\n')
        print(((len(df[col]) - df[col].isnull().sum(axis=0)) / len(df[col])) * 100)
       
            
        print('************')

ethnicity
130157
Null records 
1587


1.2192966955292455
Non Null records
128570


98.78070330447075
************
gender
130157
Null records 
66


0.05070799111841853
Non Null records
130091


99.94929200888159
************
hospital_admit_source
130157
Null records 
33198


25.50611953256452
Non Null records
96959


74.49388046743547
************
icu_admit_source
130157
Null records 
240


0.18439269497606736
Non Null records
129917


99.81560730502393
************
icu_stay_type
130157
Null records 
0


0.0
Non Null records
130157


100.0
************
icu_type
130157
Null records 
0


0.0
Non Null records
130157


100.0
************


In [10]:
removal_category_col

[]

In [11]:
numerical_Columns=[]
for i in df.columns:
    if df[i].dtype=='int32' or df[i].dtype=='float32' or  df[i].dtype=='int64' or df[i].dtype=='float64':
        numerical_Columns.append(i)

In [12]:
numerical_Columns

['Unnamed: 0',
 'encounter_id',
 'hospital_id',
 'age',
 'bmi',
 'elective_surgery',
 'height',
 'icu_id',
 'pre_icu_los_days',
 'readmission_status',
 'weight',
 'albumin_apache',
 'apache_2_diagnosis',
 'apache_3j_diagnosis',
 'apache_post_operative',
 'arf_apache',
 'bilirubin_apache',
 'bun_apache',
 'creatinine_apache',
 'fio2_apache',
 'gcs_eyes_apache',
 'gcs_motor_apache',
 'gcs_unable_apache',
 'gcs_verbal_apache',
 'glucose_apache',
 'heart_rate_apache',
 'hematocrit_apache',
 'intubated_apache',
 'map_apache',
 'paco2_apache',
 'paco2_for_ph_apache',
 'pao2_apache',
 'ph_apache',
 'resprate_apache',
 'sodium_apache',
 'temp_apache',
 'urineoutput_apache',
 'ventilated_apache',
 'wbc_apache',
 'd1_diasbp_invasive_max',
 'd1_diasbp_invasive_min',
 'd1_diasbp_max',
 'd1_diasbp_min',
 'd1_diasbp_noninvasive_max',
 'd1_diasbp_noninvasive_min',
 'd1_heartrate_max',
 'd1_heartrate_min',
 'd1_mbp_invasive_max',
 'd1_mbp_invasive_min',
 'd1_mbp_max',
 'd1_mbp_min',
 'd1_mbp_noninvasi

In [13]:
removal_numerical_columns=[]

In [14]:
for col in numerical_Columns:
    if df[col].dtype=='int32' or df[col].dtype=='float32' or df[col].dtype=='int64' or df[col].dtype=='float64':
        print(col)
        print('Null records ')
        print(df[col].isnull().sum(axis=0))
        print(((df[col].isnull().sum(axis=0)) / len(df[col])) * 100)
        if ((df[col].isnull().sum(axis=0)) / len(df[col])) * 100 > 50:
            removal_numerical_columns.append(col)
            
        print('Non Null records')
        print(len(df[col]) - df[col].isnull().sum(axis=0))
        print(((len(df[col]) - df[col].isnull().sum(axis=0)) / len(df[col])) * 100)    
        print('************')


Unnamed: 0
Null records 
0
0.0
Non Null records
130157
100.0
************
encounter_id
Null records 
0
0.0
Non Null records
130157
100.0
************
hospital_id
Null records 
0
0.0
Non Null records
130157
100.0
************
age
Null records 
4988
3.8322948439192666
Non Null records
125169
96.16770515608073
************
bmi
Null records 
4490
3.449680001843927
Non Null records
125667
96.55031999815607
************
elective_surgery
Null records 
0
0.0
Non Null records
130157
100.0
************
height
Null records 
2077
1.5957651144387162
Non Null records
128080
98.40423488556128
************
icu_id
Null records 
0
0.0
Non Null records
130157
100.0
************
pre_icu_los_days
Null records 
0
0.0
Non Null records
130157
100.0
************
readmission_status
Null records 
0
0.0
Non Null records
130157
100.0
************
weight
Null records 
3463
2.6606329279255054
Non Null records
126694
97.3393670720745
************
albumin_apache
Null records 
78163
60.05285923922648
Non Null records
5

29703
22.820900911975535
Non Null records
100454
77.17909908802446
************
h1_temp_min
Null records 
29703
22.820900911975535
Non Null records
100454
77.17909908802446
************
d1_albumin_max
Null records 
71406
54.86143657275444
Non Null records
58751
45.13856342724556
************
d1_albumin_min
Null records 
71406
54.86143657275444
Non Null records
58751
45.13856342724556
************
d1_bilirubin_max
Null records 
76735
58.95572270411888
Non Null records
53422
41.04427729588112
************
d1_bilirubin_min
Null records 
76735
58.95572270411888
Non Null records
53422
41.04427729588112
************
d1_bun_max
Null records 
13734
10.551871970005456
Non Null records
116423
89.44812802999455
************
d1_bun_min
Null records 
13734
10.551871970005456
Non Null records
116423
89.44812802999455
************
d1_calcium_max
Null records 
16692
12.824511935585486
Non Null records
113465
87.17548806441452
************
d1_calcium_min
Null records 
16692
12.824511935585486
Non Null 

In [15]:
removal_numerical_columns

['albumin_apache',
 'bilirubin_apache',
 'fio2_apache',
 'paco2_apache',
 'paco2_for_ph_apache',
 'pao2_apache',
 'ph_apache',
 'd1_diasbp_invasive_max',
 'd1_diasbp_invasive_min',
 'd1_mbp_invasive_max',
 'd1_mbp_invasive_min',
 'd1_sysbp_invasive_max',
 'd1_sysbp_invasive_min',
 'h1_diasbp_invasive_max',
 'h1_diasbp_invasive_min',
 'h1_mbp_invasive_max',
 'h1_mbp_invasive_min',
 'h1_sysbp_invasive_max',
 'h1_sysbp_invasive_min',
 'd1_albumin_max',
 'd1_albumin_min',
 'd1_bilirubin_max',
 'd1_bilirubin_min',
 'd1_inr_max',
 'd1_inr_min',
 'd1_lactate_max',
 'd1_lactate_min',
 'h1_albumin_max',
 'h1_albumin_min',
 'h1_bilirubin_max',
 'h1_bilirubin_min',
 'h1_bun_max',
 'h1_bun_min',
 'h1_calcium_max',
 'h1_calcium_min',
 'h1_creatinine_max',
 'h1_creatinine_min',
 'h1_glucose_max',
 'h1_glucose_min',
 'h1_hco3_max',
 'h1_hco3_min',
 'h1_hemaglobin_max',
 'h1_hemaglobin_min',
 'h1_hematocrit_max',
 'h1_hematocrit_min',
 'h1_inr_max',
 'h1_inr_min',
 'h1_lactate_max',
 'h1_lactate_min',

In [16]:
len(removal_numerical_columns)

73

##Droping the Category & Numerical columns with more than 70% null records

In [17]:
df.drop(removal_numerical_columns, axis=1, inplace=True)

In [18]:
df.shape

(130157, 108)

In [19]:
test_data.drop(removal_numerical_columns, axis=1, inplace=True)

In [20]:
test_data.drop(['Unnamed: 0'],axis=1,inplace=True)


In [21]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [22]:
df[df['encounter_id'].duplicated()]

,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,...,d1_wbc_max,d1_wbc_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus


In [23]:
numerical_Columns=[]
for i in df.columns:
    if df[i].dtype=='int32' or df[i].dtype=='float32' or  df[i].dtype=='int64' or df[i].dtype=='float64':
        numerical_Columns.append(i)

In [24]:
len(numerical_Columns)

101

In [25]:
existing_cat_columns=[]

for col in df.columns:
    if df[col].dtype =='object':
        existing_cat_columns.append(col)

In [26]:
existing_cat_columns

['ethnicity',
 'gender',
 'hospital_admit_source',
 'icu_admit_source',
 'icu_stay_type',
 'icu_type']

In [27]:
## Converting the binary category columns into numerical columns

In [28]:
existing_numerical_columns=[]

for col in df.columns:
    if df[col].dtype =='int32' or df[col].dtype =='float32' or df[col].dtype =='int64' or df[col].dtype =='float64':
        existing_numerical_columns.append(col)

In [29]:
existing_numerical_columns

['encounter_id',
 'hospital_id',
 'age',
 'bmi',
 'elective_surgery',
 'height',
 'icu_id',
 'pre_icu_los_days',
 'readmission_status',
 'weight',
 'apache_2_diagnosis',
 'apache_3j_diagnosis',
 'apache_post_operative',
 'arf_apache',
 'bun_apache',
 'creatinine_apache',
 'gcs_eyes_apache',
 'gcs_motor_apache',
 'gcs_unable_apache',
 'gcs_verbal_apache',
 'glucose_apache',
 'heart_rate_apache',
 'hematocrit_apache',
 'intubated_apache',
 'map_apache',
 'resprate_apache',
 'sodium_apache',
 'temp_apache',
 'urineoutput_apache',
 'ventilated_apache',
 'wbc_apache',
 'd1_diasbp_max',
 'd1_diasbp_min',
 'd1_diasbp_noninvasive_max',
 'd1_diasbp_noninvasive_min',
 'd1_heartrate_max',
 'd1_heartrate_min',
 'd1_mbp_max',
 'd1_mbp_min',
 'd1_mbp_noninvasive_max',
 'd1_mbp_noninvasive_min',
 'd1_resprate_max',
 'd1_resprate_min',
 'd1_spo2_max',
 'd1_spo2_min',
 'd1_sysbp_max',
 'd1_sysbp_min',
 'd1_sysbp_noninvasive_max',
 'd1_sysbp_noninvasive_min',
 'd1_temp_max',
 'd1_temp_min',
 'h1_diasbp_

In [30]:
df.isnull().sum(axis=0)

encounter_id                      0
hospital_id                       0
age                            4988
bmi                            4490
elective_surgery                  0
                               ... 
immunosuppression                 0
leukemia                          0
lymphoma                          0
solid_tumor_with_metastasis       0
diabetes_mellitus                 0
Length: 107, dtype: int64

In [31]:
existing_numerical_columns.remove('readmission_status')

In [32]:
df.drop(['readmission_status'],axis=1,inplace=True)


In [33]:
test_data.drop(['readmission_status'],axis=1,inplace=True)

In [34]:
df[existing_numerical_columns].isnull().sum(axis=0).nonzero()

C:\Users\SANDESH\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  """Entry point for launching an IPython kernel.


(array([ 2,  3,  5,  8,  9, 10, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24,
        25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42,
        43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
        60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76,
        77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91],
       dtype=int64),)

In [35]:
## readmission_status only labeled as 0
## apache_post_operative is binary
## elective_surgery is binary
## arf_apache : binary
## gcs_eyes_apache is multi nomial
## gcs_motor_apache is multinomial
## gcs_unable_apache is binary
## gcs_verbal_apache is multinomial
## hematocrit_apache is binary
## urineoutput_apache is binary
## d1_arterial_po2_min is binary
## aids is binary
## cirrhosis is binary
## hepatic_failure is binary
## immunosuppression is binary
## leukemia is binary
## lymphoma is binary
## solid_tumor_with_metastasis is binary
## diabetes_mellitus is binary

#binaryColumns=['gcs_eyes_apache','gcs_motor_apache','gcs_verbal_apache','apache_post_operative','arf_apache','gcs_unable_apache','hematocrit_apache',
 #           'urineoutput_apache','d1_arterial_po2_min', 'aids','cirrhosis','hepatic_failure','immunosuppression',
  #            'leukemia','lymphoma','solid_tumor_with_metastasis']

binaryColumns = ['elective_surgery','apache_post_operative','gcs_eyes_apache','gcs_motor_apache','gcs_unable_apache','gcs_verbal_apache','intubated_apache','ventilated_apache',
           'aids','cirrhosis','hepatic_failure','immunosuppression',
             'leukemia','lymphoma','solid_tumor_with_metastasis']

In [36]:
nonBinaryColumns = [x for x in existing_numerical_columns if x not in binaryColumns]

In [37]:
nonBinaryColumns

['encounter_id',
 'hospital_id',
 'age',
 'bmi',
 'height',
 'icu_id',
 'pre_icu_los_days',
 'weight',
 'apache_2_diagnosis',
 'apache_3j_diagnosis',
 'arf_apache',
 'bun_apache',
 'creatinine_apache',
 'glucose_apache',
 'heart_rate_apache',
 'hematocrit_apache',
 'map_apache',
 'resprate_apache',
 'sodium_apache',
 'temp_apache',
 'urineoutput_apache',
 'wbc_apache',
 'd1_diasbp_max',
 'd1_diasbp_min',
 'd1_diasbp_noninvasive_max',
 'd1_diasbp_noninvasive_min',
 'd1_heartrate_max',
 'd1_heartrate_min',
 'd1_mbp_max',
 'd1_mbp_min',
 'd1_mbp_noninvasive_max',
 'd1_mbp_noninvasive_min',
 'd1_resprate_max',
 'd1_resprate_min',
 'd1_spo2_max',
 'd1_spo2_min',
 'd1_sysbp_max',
 'd1_sysbp_min',
 'd1_sysbp_noninvasive_max',
 'd1_sysbp_noninvasive_min',
 'd1_temp_max',
 'd1_temp_min',
 'h1_diasbp_max',
 'h1_diasbp_min',
 'h1_diasbp_noninvasive_max',
 'h1_diasbp_noninvasive_min',
 'h1_heartrate_max',
 'h1_heartrate_min',
 'h1_mbp_max',
 'h1_mbp_min',
 'h1_mbp_noninvasive_max',
 'h1_mbp_noninv

In [38]:
len(nonBinaryColumns)

85

In [39]:
nonBinaryColumns.remove('encounter_id')
# 'encounter_id',
 #'hospital_id',
nonBinaryColumns.remove('hospital_id')  

In [40]:
for i in nonBinaryColumns:
    print(df[i].describe().quantile([0.10,0.15,0.25,0.50,0.75,0.85,0.90,1]))

0.10        11.776016
0.15        18.581736
0.25        43.205720
0.50        62.997551
0.75        78.500000
0.85        88.300000
0.90     37613.000000
1.00    125169.000000
Name: age, dtype: float64
0.10        12.870281
0.15        15.282580
0.25        21.409736
0.50        28.337504
0.75        41.556093
0.85        66.064397
0.90     37747.570493
1.00    125667.000000
Name: bmi, dtype: float64
0.10        99.289926
0.15       138.465000
0.25       156.175000
0.50       169.853609
0.75       182.247500
0.85       194.700500
0.90     38560.913000
1.00    128080.000000
Name: height, dtype: float64
0.10       237.581890
0.15       310.396851
0.25       396.314961
0.50       657.714172
0.75      1004.500000
0.85      1103.900000
0.90     39824.800000
1.00    130157.000000
Name: icu_id, dtype: float64
0.10        -0.042917
0.15         0.051319
0.25         0.128125
0.50         0.631772
0.75        45.770947
0.85       166.970656
0.90     39170.039444
1.00    130157.000000
Name: pre_

Name: d1_temp_min, dtype: float64
0.10        31.461922
0.15        38.250000
0.25        55.750000
0.50        74.713047
0.75       100.250000
0.85       140.150000
0.90     37489.100000
1.00    124630.000000
Name: h1_diasbp_max, dtype: float64
0.10        20.291604
0.15        23.450000
0.25        43.750000
0.50        61.747424
0.75        83.000000
0.85       111.000000
0.90     37468.100000
1.00    124630.000000
Name: h1_diasbp_min, dtype: float64
0.10        31.492840
0.15        38.300000
0.25        56.500000
0.50        74.950794
0.75       101.250000
0.85       141.150000
0.90     35746.200000
1.00    118818.000000
Name: h1_diasbp_noninvasive_max, dtype: float64
0.10        20.315996
0.15        23.500000
0.25        44.500000
0.50        62.447588
0.75        83.250000
0.85       111.950000
0.90     35725.200000
1.00    118818.000000
Name: h1_diasbp_noninvasive_min, dtype: float64
0.10        38.734976
0.15        47.550000
0.25        69.250000
0.50        91.228691
0.75  

In [41]:
df = df.loc[ (df['age'] < df['age'].quantile(0.85)) ]

In [42]:
df.shape

(105687, 106)

In [43]:
for i in nonBinaryColumns:
    print(df[i].describe().quantile([0.10,0.15,0.25,0.50,0.75,0.85,0.90,1]))

0.10        10.562358
0.15        16.834629
0.25        41.272271
0.50        59.465918
0.75        72.250000
0.85        78.550000
0.90     31761.400000
1.00    105687.000000
Name: age, dtype: float64
0.10        12.949839
0.15        15.305160
0.25        21.748438
0.50        29.021922
0.75        42.199549
0.85        66.107294
0.90     30667.570493
1.00    102067.000000
Name: bmi, dtype: float64
0.10        99.252945
0.15       138.470000
0.25       156.250000
0.50       170.285520
0.75       182.247500
0.85       194.700500
0.90     31340.213000
1.00    104011.000000
Name: height, dtype: float64
0.10       237.773266
0.15       310.706575
0.25       397.133309
0.50       661.676720
0.75      1007.500000
0.85      1104.100000
0.90     32483.800000
1.00    105687.000000
Name: icu_id, dtype: float64
0.10        -0.043889
0.15         0.049896
0.25         0.126215
0.50         0.621782
0.75        45.810251
0.85       166.973276
0.90     31829.039444
1.00    105687.000000
Name: pre_

0.10         2.108458
0.15         7.711479
0.25        73.503021
0.50        98.577878
0.75       100.000000
0.85       100.000000
0.90     30236.500000
1.00    100555.000000
Name: h1_spo2_max, dtype: float64
0.10         4.424091
0.15        10.704123
0.25        72.080032
0.50        96.197698
0.75        99.250000
0.85        99.950000
0.90     30236.500000
1.00    100555.000000
Name: h1_spo2_min, dtype: float64
0.10        60.646464
0.15        76.900000
0.25       103.500000
0.50       131.458000
0.75       167.500000
0.85       219.300000
0.90     30499.000000
1.00    101143.000000
Name: h1_sysbp_max, dtype: float64
0.10        44.871279
0.15        55.250000
0.25        86.750000
0.50       114.942739
0.75       148.250000
0.85       190.950000
0.90     30478.700000
1.00    101143.000000
Name: h1_sysbp_min, dtype: float64
0.10       60.679352
0.15       76.900000
0.25      103.500000
0.50      131.354900
0.75      167.500000
0.85      219.300000
0.90    29033.500000
1.00    962

In [44]:
df = df.loc[ (df['bmi'] < df['bmi'].quantile(0.85)) ]

In [45]:
for i in nonBinaryColumns:
    print(df[i].describe().quantile([0.15,0.25,0.50,0.75,0.85,0.90,1]))

0.15       17.008489
0.25       41.318023
0.50       59.581586
0.75       72.250000
0.85       78.550000
0.90    26081.800000
1.00    86755.000000
Name: age, dtype: float64
0.15       15.271504
0.25       21.243601
0.50       27.032335
0.75       32.482391
0.85       37.043800
0.90    26052.658730
1.00    86755.000000
Name: bmi, dtype: float64
0.15      138.47000
0.25      156.25000
0.50      170.49472
0.75      182.39750
0.85      194.71050
0.90    26163.41300
1.00    86755.00000
Name: height, dtype: float64
0.15      305.731247
0.25      398.771381
0.50      668.905031
0.75     1009.000000
0.85     1104.200000
0.90    26804.200000
1.00    86755.000000
Name: icu_id, dtype: float64
0.15        0.052708
0.25        0.129514
0.50        0.621735
0.75       45.837193
0.85      166.975072
0.90    26149.439444
1.00    86755.000000
Name: pre_icu_los_days, dtype: float64
0.15       39.990000
0.25       59.450000
0.50       79.057293
0.75      109.825000
0.85      162.255000
0.90    26142.7000

0.15        6.285000
0.25        7.475000
0.50        8.382746
0.75        9.300000
0.85       10.700000
0.90    22782.660000
1.00    75917.000000
Name: d1_calcium_max, dtype: float64
0.15        5.610000
0.25        7.150000
0.50        8.167912
0.75        9.100000
0.85       10.220000
0.90    22782.310000
1.00    75917.000000
Name: d1_calcium_min, dtype: float64
0.15        0.731000
0.25        0.885000
0.50        1.427690
0.75        3.970423
0.85       10.634028
0.90    23443.777000
1.00    78120.000000
Name: d1_creatinine_max, dtype: float64
0.15        0.710000
0.25        0.850000
0.50        1.298881
0.75        3.526928
0.85        9.510502
0.90    23442.956530
1.00    78120.000000
Name: d1_creatinine_min, dtype: float64
0.15       89.798374
0.25      109.104835
0.50      160.718049
0.75      301.250000
0.85      590.350000
0.90    24800.000000
1.00    81241.000000
Name: d1_glucose_max, dtype: float64
0.15       39.687621
0.25       76.759900
0.50      109.420225
0.75      1

In [46]:
df.shape

(86755, 106)

In [47]:
df = df.loc[ (df['height'] < df['height'].quantile(0.85)) ]

In [48]:
df.shape

(73255, 106)

In [49]:
#df = df.loc[(df['weight'] < df['weight'].quantile(0.85))]

In [50]:
df.shape

(73255, 106)

## Imputing Mode VALUE for all the category Columns

In [51]:
existing_cat_columns

['ethnicity',
 'gender',
 'hospital_admit_source',
 'icu_admit_source',
 'icu_stay_type',
 'icu_type']

In [52]:
test_data.isnull().sum(axis=0)

encounter_id                      0
hospital_id                       0
age                               0
bmi                            1015
elective_surgery                  0
                               ... 
hepatic_failure                   0
immunosuppression                 0
leukemia                          0
lymphoma                          0
solid_tumor_with_metastasis       0
Length: 105, dtype: int64

In [53]:
for col in existing_cat_columns:
     df[col].fillna(df[col].mode()[0],inplace=True)

In [54]:
existing_cat_columns1 = existing_cat_columns

In [55]:
#existing_cat_columns1.remove('diabetes_mellitus')

In [56]:
existing_cat_columns1

['ethnicity',
 'gender',
 'hospital_admit_source',
 'icu_admit_source',
 'icu_stay_type',
 'icu_type']

In [57]:
test_data[existing_cat_columns1].isnull().sum(axis=0)

ethnicity                 204
gender                      5
hospital_admit_source    2733
icu_admit_source           25
icu_stay_type               0
icu_type                    0
dtype: int64

In [58]:
for col in existing_cat_columns1:
     test_data[col].fillna(test_data[col].mode()[0],inplace=True)

In [59]:
df[existing_cat_columns].isnull().sum(axis=0)

ethnicity                0
gender                   0
hospital_admit_source    0
icu_admit_source         0
icu_stay_type            0
icu_type                 0
dtype: int64

In [60]:
for i in existing_numerical_columns:
    print(df[i].value_counts())

272379    1
203638    1
246637    1
207728    1
205681    1
         ..
218496    1
175487    1
173438    1
167291    1
264193    1
Name: encounter_id, Length: 73255, dtype: int64
19     2173
118    1942
161    1776
86     1747
188    1709
       ... 
38        3
25        2
93        2
95        2
130       1
Name: hospital_id, Length: 204, dtype: int64
67.0    2297
71.0    2202
72.0    2173
68.0    2143
66.0    2096
        ... 
20.0     356
18.0     271
17.0     144
16.0      72
0.0       10
Name: age, Length: 65, dtype: int64
14.844926    415
28.690874    103
27.359332     94
24.208110     87
29.049732     87
            ... 
30.834984      1
35.302111      1
22.195039      1
25.513716      1
25.429106      1
Name: bmi, Length: 22933, dtype: int64
0    58294
1    14961
Name: elective_surgery, dtype: int64
177.8    5436
167.6    4926
172.7    4813
160.0    4491
165.1    4418
         ... 
161.1       1
173.8       1
147.4       1
161.4       1
176.7       1
Name: height, Length: 334

122.0    1072
119.0    1041
118.0    1036
128.0    1034
127.0    1015
         ... 
213.0      23
216.0      23
217.0      21
221.0      17
218.0      16
Name: h1_sysbp_noninvasive_max, Length: 149, dtype: int64
108.0    1100
106.0    1099
110.0    1088
112.0    1080
105.0    1079
         ... 
54.0       36
192.0      29
191.0      25
194.0      21
193.0      13
Name: h1_sysbp_noninvasive_min, Length: 143, dtype: int64
36.800000    4944
36.700000    4912
36.600000    4726
36.900000    4144
36.400000    3902
             ... 
37.833333       1
35.350000       1
35.750000       1
37.388889       1
36.250000       1
Name: h1_temp_max, Length: 321, dtype: int64
36.70000    4879
36.60000    4850
36.80000    4708
36.40000    4256
36.90000    3847
            ... 
33.88000       1
38.55864       1
35.35000       1
35.16700       1
34.66700       1
Name: h1_temp_min, Length: 337, dtype: int64
13.0    3265
11.0    3215
12.0    3177
14.0    3107
10.0    3070
        ... 
24.1       1
24.3      

In [61]:
for col in existing_cat_columns:
     df[col].fillna(df[col].mode()[0],inplace=True)

In [62]:
for col in binaryColumns:
    df[col].fillna(df[col].mode()[0],inplace=True)

In [63]:
for col in binaryColumns:
    test_data[col].fillna(test_data[col].mode()[0],inplace=True)

In [64]:
for i in nonBinaryColumns:
    df[i].fillna(df[i].mean(),inplace=True)

In [65]:
#nonBinaryColumns

In [66]:
nonBinaryColumns1 = nonBinaryColumns

In [67]:
nonBinaryColumns1.remove('diabetes_mellitus')

In [68]:
for i in nonBinaryColumns:
    test_data[i].fillna(test_data[i].mean(),inplace=True)

In [69]:
df[existing_numerical_columns].isnull().sum(axis=0).nonzero()

C:\Users\SANDESH\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  """Entry point for launching an IPython kernel.


(array([], dtype=int64),)

In [70]:
df[existing_cat_columns].isnull().sum(axis=0).nonzero()

C:\Users\SANDESH\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  """Entry point for launching an IPython kernel.


(array([], dtype=int64),)

In [71]:
## prediction variable : diabetes_mellitus

In [72]:
df['diabetes_mellitus'].value_counts()

0    58515
1    14740
Name: diabetes_mellitus, dtype: int64

In [73]:
#Prediction value is quite imbalance . with only 28151 as 1 and over 1 lac records as 0

In [74]:
existing_cat_columns

['ethnicity',
 'gender',
 'hospital_admit_source',
 'icu_admit_source',
 'icu_stay_type',
 'icu_type']

In [75]:
len(df.columns)

106

In [76]:
target= df['diabetes_mellitus']

In [77]:
df_woe=df

In [78]:
df_woe.drop(['diabetes_mellitus'],axis=1,inplace=True)

In [79]:
#WOE and IV

In [80]:
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

max_bin = 10
force_bin = 3

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [81]:
final_iv, IV  = data_vars(df_woe,target)

C:\Users\SANDESH\Anaconda3\lib\site-packages\pandas\core\series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [82]:
# Sorting the column in Descending order 
IV = IV.sort_values('IV',ascending=False)

In [83]:
iv_columns = IV.loc[(IV['IV'] > 0.020) ,'VAR_NAME']

In [84]:
iv_columns

20                d1_glucose_max
60                glucose_apache
10                    d1_bun_max
7                     bun_apache
14             d1_creatinine_max
11                    d1_bun_min
15             d1_creatinine_min
9              creatinine_apache
6                            bmi
104                       weight
0                            age
5                     arf_apache
36              d1_potassium_max
44                  d1_sysbp_max
46      d1_sysbp_noninvasive_max
26             d1_hemaglobin_max
41                 d1_sodium_min
28             d1_hematocrit_max
62                 h1_diasbp_min
27             d1_hemaglobin_min
90                      icu_type
54                     ethnicity
29             d1_hematocrit_min
64     h1_diasbp_noninvasive_min
83             hematocrit_apache
21                d1_glucose_min
Name: VAR_NAME, dtype: object

In [86]:
encounter_id_train= df_woe['encounter_id']

In [87]:
df_woe = df_woe[iv_columns]

In [88]:
encounter_id_test = test_data['encounter_id']

In [89]:
test_data_woe = test_data[iv_columns]

In [90]:
existing_cat_columns=[]

for col in df_woe.columns:
    if df_woe[col].dtype =='object':
        existing_cat_columns.append(col)

In [91]:
cat=[]
val=0
dict={}
for col in existing_cat_columns:
    dict[col] = pd.get_dummies(df_woe[col],prefix=col,drop_first=True)

In [92]:
cat_df = pd.concat(dict.values(), axis=1)

In [93]:
cat_df.shape

(73255, 12)

In [94]:
df_woe.drop(existing_cat_columns, axis=1, inplace=True)

C:\Users\SANDESH\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [95]:
len(df_woe.columns)

24

In [96]:
## Concating the dummy columns to main dataframe 

In [97]:
df_rf = pd.concat([df_woe, cat_df], axis=1)

In [98]:
len(df_rf.columns)

36

In [99]:
df_rf.head()

,d1_glucose_max,glucose_apache,d1_bun_max,bun_apache,d1_creatinine_max,d1_bun_min,d1_creatinine_min,creatinine_apache,bmi,weight,...,icu_type_Cardiac ICU,icu_type_MICU,icu_type_Med-Surg ICU,icu_type_Neuro ICU,icu_type_SICU,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown
0,168.000000,168.000000,31.000000,31.000000,2.510000,30.000000,2.230000,2.510000,22.732803,73.9,...,0,0,0,0,0,0,1,0,0,0
1,145.000000,145.000000,11.000000,9.000000,0.710000,9.000000,0.560000,0.560000,27.421875,70.2,...,0,0,1,0,0,0,1,0,0,0
2,174.057595,159.705603,23.287333,23.423068,1.436529,21.175331,1.299785,1.419949,31.952749,95.3,...,0,0,1,0,0,0,1,0,0,0
9,134.000000,134.000000,10.000000,10.000000,0.830000,10.000000,0.830000,0.830000,25.707702,79.0,...,0,0,0,0,0,0,1,0,0,0
10,158.000000,159.705603,14.000000,23.423068,0.900000,14.000000,0.900000,1.419949,28.257052,67.8,...,0,0,1,0,0,0,0,1,0,0


In [100]:
#df_rf.set_index('encounter_id')

In [153]:
hospital_id= df['hospital_id']

In [154]:
df_rf.drop(['hospital_id','icu_id'],axis=1,inplace=True)


In [101]:
existing_cat_columns_test=[]

for col in test_data_woe.columns:
    if test_data_woe[col].dtype =='object':
        existing_cat_columns_test.append(col)

cat=[]
val=0
dict={}
for col in existing_cat_columns:
    dict[col] = pd.get_dummies(test_data_woe[col],prefix=col,drop_first=True)

cat_test_df = pd.concat(dict.values(), axis=1)
test_data_woe.drop(existing_cat_columns_test, axis=1, inplace=True)
test_rf = pd.concat([test_data_woe, cat_test_df], axis=1)


In [155]:
#df_rf.drop(['ethnicity'], axis=1,inplace=True)

In [162]:
#from sklearn.model_selection import train_test_split

#X_train,X_test,y_train,y_test =  train_test_split(df_rf,target,test_size=0.3,random_state=42)

In [103]:
#nonBinaryColumns.remove('diabetes_mellitus')

In [106]:
nonBinaryColumns=[]

In [107]:
for i in df_rf.columns:
    if df_rf[i].dtype=='float':
        nonBinaryColumns.append(i)

In [106]:
nonBinaryColumns.remove('icu_id')

ValueError: list.remove(x): x not in list

In [108]:
nonBinaryColumns1=[]

In [109]:
for i in test_rf.columns:
    if test_rf[i].dtype=='float':
        nonBinaryColumns1.append(i)

In [110]:
#nonBinaryColumns1

In [111]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

df_rf[nonBinaryColumns] = scaler.fit_transform(df_rf[nonBinaryColumns])
test_rf[nonBinaryColumns1] = scaler.fit_transform(test_rf[nonBinaryColumns1])

In [227]:
# oversampling process 

In [112]:
from imblearn.over_sampling import SMOTE

Using TensorFlow backend.
C:\Users\SANDESH\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\SANDESH\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\SANDESH\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\SANDESH\Anaconda3\lib\site-packages\tensorflow\python\framework\d

In [113]:

sm = SMOTE(random_state=2)
X_train_log, y_train_log = sm.fit_sample(df_rf, target)

In [114]:
from collections import Counter

In [115]:
Counter(y_train_log)

Counter({1: 58515, 0: 58515})

In [116]:
logisiticRegression = LogisticRegression()
clf = logisiticRegression.fit(X_train_log,y_train_log)

In [117]:
def generate_model_report(y_actual, y_predicted):
    print("Accuracy = " , accuracy_score(y_actual, y_predicted))
    print('\n')
    print("Precision = " ,precision_score(y_actual, y_predicted))
    print('\n')
    print("Recall = " ,recall_score(y_actual, y_predicted))
    print('\n')
    print("F1 Score = " ,f1_score(y_actual, y_predicted)) 
    print('\n')
    pass

In [118]:
generate_model_report(y_train_log, clf.predict(X_train_log))

Accuracy =  0.7813381184311715


Precision =  0.7959550561797752


Recall =  0.7566435956592327


F1 Score =  0.7758016471000525




In [119]:
from sklearn import metrics
def evaluate_metric_of_model(converted,predicted):

    # Confusion matrix 
    confusion = metrics.confusion_matrix(converted,predicted)
    print('--'*20)
    print('confusion_matrix')
    print(confusion)


    # Let's check the overall accuracy.
    print('--'*20)
    print('accuracy')

    print(metrics.accuracy_score(converted,predicted).round(2))

    TP = confusion[1,1] # true positive 
    TN = confusion[0,0] # true negatives
    FP = confusion[0,1] # false positives
    FN = confusion[1,0] # false negatives


    # Let's see the sensitivity of our logistic regression model
    print('--'*20)
    print('sensitivity')
    print((TP / float(TP+FN)).round(2))


    # Let us calculate specificity
    print('--'*20)
    print('specificity')
    print((TN / float(TN+FP)).round(2))


    # Calculate false postive rate 
    print('--'*20)
    print('False Positive Rate')
    print((FP/ float(TN+FP)).round(2))


    # positive predictive value 
    print('--'*20)
    print('Positive predictive value')
    print ((TP / float(TP+FP)).round(2))
    # Negative predictive value
    print('--'*20)
    print('Negative predictive value')
    print ((TN / float(TN+ FN)).round(2))
    generate_model_report(converted,predicted)
	
	

In [120]:
from lightgbm import LGBMClassifier
from sklearn import metrics
lgbmc = LGBMClassifier()
lgbmc.fit(X_train_log, y_train_log)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [121]:
lgbmc.score(X_train_log, y_train_log)

0.8895582329317269

In [122]:
ytest_pred_lgbmc = lgbmc.predict(X_test)

NameError: name 'X_test' is not defined

In [241]:
evaluate_metric_of_model(y_test ,ytest_pred_lgbmc)

----------------------------------------
confusion_matrix
[[14349   746]
 [ 2187  1393]]
----------------------------------------
accuracy
0.84
----------------------------------------
sensitivity
0.39
----------------------------------------
specificity
0.95
----------------------------------------
False Positive Rate
0.05
----------------------------------------
Positive predictive value
0.65
----------------------------------------
Negative predictive value
0.87
Accuracy =  0.8429451137884872


Precision =  0.6512388966806919


Recall =  0.38910614525139664


F1 Score =  0.48714810281517745




In [123]:
from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor

In [124]:
CCS = CatBoostClassifier(eval_metric="AUC", depth=10, l2_leaf_reg= 9, learning_rate= 0.15,iterations=500)

In [125]:
CCS.fit(X_train_log,y_train_log)

0:	total: 358ms	remaining: 2m 58s
1:	total: 618ms	remaining: 2m 33s
2:	total: 913ms	remaining: 2m 31s
3:	total: 1.18s	remaining: 2m 25s
4:	total: 1.42s	remaining: 2m 20s
5:	total: 1.69s	remaining: 2m 18s
6:	total: 1.93s	remaining: 2m 16s
7:	total: 2.18s	remaining: 2m 14s
8:	total: 2.44s	remaining: 2m 13s
9:	total: 2.68s	remaining: 2m 11s
10:	total: 2.92s	remaining: 2m 10s
11:	total: 3.25s	remaining: 2m 12s
12:	total: 3.57s	remaining: 2m 13s
13:	total: 3.85s	remaining: 2m 13s
14:	total: 4.17s	remaining: 2m 14s
15:	total: 4.44s	remaining: 2m 14s
16:	total: 4.74s	remaining: 2m 14s
17:	total: 5.02s	remaining: 2m 14s
18:	total: 5.35s	remaining: 2m 15s
19:	total: 5.68s	remaining: 2m 16s
20:	total: 5.95s	remaining: 2m 15s
21:	total: 6.21s	remaining: 2m 14s
22:	total: 6.45s	remaining: 2m 13s
23:	total: 6.7s	remaining: 2m 12s
24:	total: 6.94s	remaining: 2m 11s
25:	total: 7.21s	remaining: 2m 11s
26:	total: 7.44s	remaining: 2m 10s
27:	total: 7.71s	remaining: 2m 9s
28:	total: 7.95s	remaining: 2m 9

233:	total: 1m 5s	remaining: 1m 14s
234:	total: 1m 5s	remaining: 1m 13s
235:	total: 1m 5s	remaining: 1m 13s
236:	total: 1m 6s	remaining: 1m 13s
237:	total: 1m 6s	remaining: 1m 13s
238:	total: 1m 6s	remaining: 1m 12s
239:	total: 1m 7s	remaining: 1m 12s
240:	total: 1m 7s	remaining: 1m 12s
241:	total: 1m 7s	remaining: 1m 12s
242:	total: 1m 8s	remaining: 1m 12s
243:	total: 1m 8s	remaining: 1m 11s
244:	total: 1m 8s	remaining: 1m 11s
245:	total: 1m 9s	remaining: 1m 11s
246:	total: 1m 9s	remaining: 1m 10s
247:	total: 1m 9s	remaining: 1m 10s
248:	total: 1m 9s	remaining: 1m 10s
249:	total: 1m 10s	remaining: 1m 10s
250:	total: 1m 10s	remaining: 1m 9s
251:	total: 1m 10s	remaining: 1m 9s
252:	total: 1m 10s	remaining: 1m 9s
253:	total: 1m 11s	remaining: 1m 8s
254:	total: 1m 11s	remaining: 1m 8s
255:	total: 1m 11s	remaining: 1m 8s
256:	total: 1m 11s	remaining: 1m 7s
257:	total: 1m 12s	remaining: 1m 7s
258:	total: 1m 12s	remaining: 1m 7s
259:	total: 1m 12s	remaining: 1m 7s
260:	total: 1m 12s	remainin

463:	total: 2m 10s	remaining: 10.1s
464:	total: 2m 11s	remaining: 9.86s
465:	total: 2m 11s	remaining: 9.59s
466:	total: 2m 11s	remaining: 9.32s
467:	total: 2m 12s	remaining: 9.04s
468:	total: 2m 12s	remaining: 8.77s
469:	total: 2m 12s	remaining: 8.48s
470:	total: 2m 13s	remaining: 8.2s
471:	total: 2m 13s	remaining: 7.91s
472:	total: 2m 13s	remaining: 7.63s
473:	total: 2m 14s	remaining: 7.35s
474:	total: 2m 14s	remaining: 7.07s
475:	total: 2m 14s	remaining: 6.78s
476:	total: 2m 14s	remaining: 6.5s
477:	total: 2m 15s	remaining: 6.22s
478:	total: 2m 15s	remaining: 5.93s
479:	total: 2m 15s	remaining: 5.65s
480:	total: 2m 15s	remaining: 5.37s
481:	total: 2m 16s	remaining: 5.08s
482:	total: 2m 16s	remaining: 4.8s
483:	total: 2m 16s	remaining: 4.51s
484:	total: 2m 16s	remaining: 4.23s
485:	total: 2m 17s	remaining: 3.95s
486:	total: 2m 17s	remaining: 3.67s
487:	total: 2m 17s	remaining: 3.38s
488:	total: 2m 17s	remaining: 3.1s
489:	total: 2m 18s	remaining: 2.82s
490:	total: 2m 18s	remaining: 2.

In [126]:
CCS.score(X_train_log,y_train_log)

0.9658207297274203

In [127]:
test_data_woe

,d1_glucose_max,glucose_apache,d1_bun_max,bun_apache,d1_creatinine_max,d1_bun_min,d1_creatinine_min,creatinine_apache,bmi,weight,...,d1_sysbp_noninvasive_max,d1_hemaglobin_max,d1_sodium_min,d1_hematocrit_max,h1_diasbp_min,d1_hemaglobin_min,d1_hematocrit_min,h1_diasbp_noninvasive_min,hematocrit_apache,d1_glucose_min
0,104.000000,97.000000,44.000000,44.000000,1.490000,40.00000,1.440000,1.490000,29.116463,83.478729,...,149.000,14.500000,130.000000,42.800000,59.0000,14.200000,39.900000,59.000000,39.900000,97.000000
1,102.000000,73.000000,19.000000,19.000000,1.000000,19.00000,1.000000,0.920000,29.116463,83.478729,...,181.000,11.516973,144.000000,34.647799,65.0000,10.915915,32.837469,65.000000,32.888448,73.000000
2,141.000000,84.000000,24.000000,24.000000,0.820000,16.00000,0.770000,0.770000,29.116463,83.478729,...,146.000,11.800000,137.000000,36.600000,97.0000,11.500000,34.800000,97.000000,34.800000,84.000000
3,84.000000,161.399236,27.000000,25.110221,1.550000,27.00000,1.550000,1.397522,29.116463,83.478729,...,145.000,10.800000,136.000000,34.300000,62.3421,10.800000,34.300000,62.809881,32.888448,84.000000
4,159.000000,99.000000,26.000000,26.000000,1.500000,25.00000,1.300000,1.500000,29.116463,83.478729,...,194.000,10.800000,133.000000,34.200000,66.0000,10.700000,33.000000,66.000000,33.000000,99.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10229,96.000000,161.399236,45.000000,25.110221,3.230000,45.00000,3.230000,1.397522,37.500000,108.600000,...,173.000,10.100000,137.000000,30.000000,74.0000,9.500000,27.000000,74.000000,29.000000,96.000000
10230,94.000000,94.000000,33.000000,33.000000,1.150000,33.00000,1.150000,1.150000,32.100000,82.300000,...,228.235,9.400000,139.000000,30.000000,62.3421,8.900000,27.000000,62.809881,27.000000,94.000000
10231,150.000000,161.399236,23.000000,25.110221,1.090000,23.00000,1.090000,1.397522,22.700000,62.000000,...,185.000,9.900000,141.000000,30.000000,50.0000,9.900000,30.000000,50.000000,32.888448,150.000000
10232,98.000000,161.399236,16.000000,25.110221,0.920000,16.00000,0.920000,1.397522,19.900000,50.900000,...,150.000,11.900000,139.000000,37.000000,61.0000,11.900000,37.000000,61.000000,32.888448,98.000000


In [128]:
test_rf

,d1_glucose_max,glucose_apache,d1_bun_max,bun_apache,d1_creatinine_max,d1_bun_min,d1_creatinine_min,creatinine_apache,bmi,weight,...,icu_type_Cardiac ICU,icu_type_MICU,icu_type_Med-Surg ICU,icu_type_Neuro ICU,icu_type_SICU,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown
0,-8.733917e-01,-7.816991e-01,9.902301e-01,1.032953e+00,5.955470e-02,9.669778e-01,1.288569e-01,7.528456e-02,1.346078e-15,-1.512088e-14,...,0,0,1,0,0,0,1,0,0,0
1,-8.982331e-01,-1.073019e+00,-3.118452e-01,-3.341261e-01,-3.220211e-01,-2.287558e-01,-2.581700e-01,-3.887407e-01,1.346078e-15,-1.512088e-14,...,0,0,1,0,0,0,1,0,0,0
2,-4.138247e-01,-9.394973e-01,-5.143012e-02,-6.071040e-02,-4.621918e-01,-3.995749e-01,-4.604794e-01,-5.108526e-01,1.346078e-15,-1.512088e-14,...,0,0,1,0,0,0,1,0,0,0
3,-1.121806e+00,6.899841e-16,1.048189e-01,1.165641e-15,1.062783e-01,2.267618e-01,2.256136e-01,7.230478e-16,1.346078e-15,-1.512088e-14,...,0,0,1,0,0,0,1,0,0,0
4,-1.902515e-01,-7.574224e-01,5.273590e-02,4.865589e-02,6.734196e-02,1.128824e-01,5.711997e-03,8.342535e-02,1.346078e-15,-1.512088e-14,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10229,-9.727575e-01,6.899841e-16,1.042313e+00,1.165641e-15,1.414538e+00,1.251676e+00,1.703353e+00,7.230478e-16,1.058805e+00,1.069198e+00,...,0,0,1,0,0,0,1,0,0,0
10230,-9.975990e-01,-8.181141e-01,4.173170e-01,4.314379e-01,-2.052122e-01,5.683999e-01,-1.262290e-01,-2.015024e-01,3.768081e-01,-5.016843e-02,...,0,0,1,0,0,0,1,0,0,0
10231,-3.020381e-01,6.899841e-16,-1.035131e-01,1.165641e-15,-2.519357e-01,-9.970374e-04,-1.790054e-01,7.230478e-16,-8.103719e-01,-9.141665e-01,...,0,0,1,0,0,0,1,0,0,0
10232,-9.479160e-01,6.899841e-16,-4.680942e-01,1.165641e-15,-3.843191e-01,-3.995749e-01,-3.285385e-01,7.230478e-16,-1.164000e+00,-1.386599e+00,...,0,0,1,0,0,0,1,0,0,0


In [133]:
test_pred = CCS.predict_proba(test_rf)[:,0]

In [134]:
encounter_id_test

0        144740
1        141990
2        142038
3        138628
4        141682
          ...  
10229    143750
10230    143813
10231    137126
10232    135652
10233    136852
Name: encounter_id, Length: 10234, dtype: int64

In [135]:
submission =pd.DataFrame({'encounter_id' : encounter_id_test, 'diabetes_mellitus' : test_pred})

In [136]:
submission.to_csv('submission.csv', index=False)